In [ ]:
import pandas as pd
import numpy as np

#Importing libraries  for visualisation of data
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 12})

#Importing sklearn libraries for modelling and evaluation

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor as knnr
from sklearn.metrics import mean_squared_error as mse



from random import randint #To generate random numbers in a given range

#Importing datetime module
from time import time
from datetime import date, timedelta #For creating additional time based features


from sklearn.preprocessing import MinMaxScaler ## Importing the MinMax Scaler

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_curve, roc_auc_score, auc, mean_squared_log_error


#importing all the important libraries
#Importing XGBoost module
import xgboost as xgb 
from xgboost import plot_importance
from xgboost import XGBRegressor #For modelling train data to predict Sales

pd.set_option('display.max_columns', None)  #To display all the columns in dataset

In [ ]:
ppf = pd.read_csv('processedflights15july.csv')

# X = ppf.drop(["Arrival Delay (Minutes)", "Delayed?","Taxi-In time (Minutes)", "xa", "ya", "Departure Delay (Minutes)"],axis=1)
X = ppf.drop(["Arrival Delay (Minutes)", "Delayed?"],axis=1)
y = ppf["Arrival Delay (Minutes)"]
y1 = ppf["Delayed?"]
y2 = ppf["Departure Delay (Minutes)"]

scaler = MinMaxScaler()
Xs = scaler.fit_transform(X)

train_x,test_x,train_y,test_y = train_test_split(Xs,y1, test_size=0.2, random_state = 50, stratify=y1)
#for classification
train_x1, val_x, train_y1, val_y = train_test_split(train_x, train_y, test_size = 0.2 , random_state = 50, stratify = train_y)

print('training data    ',train_x.shape,train_y.shape)
print('validation data  ',val_x.shape,val_y.shape)
print('test data        ',test_x.shape,test_y.shape)

train_xr,test_xr,train_yr,test_yr = train_test_split(Xs,y, test_size=0.2, random_state = 50, stratify=y1)
#for arrival regression
train_x1r, val_xr, train_y1r, val_yr = train_test_split(train_xr, train_yr, test_size = 0.2 , random_state = 50, stratify = train_y)

print('training data    ',train_xr.shape,train_yr.shape)
print('validation data  ',val_xr.shape,val_yr.shape)
print('test data        ',test_xr.shape,test_yr.shape)

train_xr1,test_xr1,train_yr1,test_yr1 = train_test_split(Xs,y2, test_size=0.2, random_state = 50, stratify=y1)
#for delay regression
train_x1r1, val_xr1, train_y1r1, val_yr1 = train_test_split(train_xr1, train_yr1, test_size = 0.2 , random_state = 50, stratify = train_y)

print('training data    ',train_xr1.shape,train_yr1.shape)
print('validation data  ',val_xr1.shape,val_yr1.shape)
print('test data        ',test_xr1.shape,test_yr1.shape)

XGBOOST

In [ ]:
params0 = {"objective": "reg:linear",          
        "booster" : "gbtree",
          "eta": 0.05,
          "max_depth": 12,
          "subsample": 0.95,
          "colsample_bytree": 0.9,
          "silent": 1,
          "seed": 50}

#   
num_round0 = 600


dtrain = xgb.DMatrix(train_xr, train_yr)
dvalid = xgb.DMatrix(test_xr, test_yr)
# We will assign the "evals" parameter the value watchlist which is a requirement for early_stopping_rounds
evals0 = [(dtrain, 'train'), (dvalid, 'val')] 

In [ ]:
xgbmodel = xgb.train(params0, dtrain, num_round0, evals=evals0, early_stopping_rounds=30, verbose_eval=True)

In [ ]:
ypreds = xgbmodel.predict(xgb.DMatrix(test_xr))

In [ ]:
print(ypreds[:5])
print(test_yr[:5])

In [ ]:
error = np.sqrt(mse(test_yr, ypreds))
error